In [5]:
from wx_data.noaa_api_v2 import NOAAData
import pandas as pd

In [6]:
token = 'HPlApRKyCHekmCNBeYOiPtNwWMBjybPt'

noaa_data = NOAAData(token=token)

In [7]:
stations = noaa_data.stations()

Metadata for stations: 
 {'resultset': {'offset': 1, 'count': 143928, 'limit': 1000}}


In [10]:
stations = pd.DataFrame(stations)

In [15]:
stations.to_csv('all_stations.csv')

In [106]:
nm_locations = pd.read_csv('info/nm_locations.csv')
# nm_stations.merge(how='left', right=stations, left_on='id', right_on='id')
all_stations = pd.read_csv('info/all_stations.csv',index_col="idx", parse_dates=['mindate','maxdate'])
nm_stations = all_stations[all_stations.name.str.contains("NM US")]
nm_stations = nm_stations[~nm_stations.elevationUnit.isna()]
[_, my_lat, my_lon, my_elev_m, _] = pd.read_csv("info/gps_coords.csv").iloc[0].to_list()
nm_stations


,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
idx,,,,,,,,,
10973,1874.5,1931-01-01,2015-11-01,36.30270,"ABBOTT 1 SE, NM US",0.8499,COOP:290022,METERS,-104.24970
10974,1800.1,1942-07-01,1948-04-01,36.16667,"ABIQUIU, NM US",0.9284,COOP:290039,METERS,-106.18333
10975,1944.6,1957-06-01,2015-11-01,36.24020,"ABIQUIU DAM, NM US",0.9943,COOP:290041,METERS,-106.42770
10976,2007.1,1941-01-01,1953-04-01,35.05000,"ACOMITA AIRPORT, NM US",0.9328,COOP:290074,METERS,-107.71667
10977,2396.9,1948-06-01,1949-08-01,34.46667,"ADAMS DIGGINGS, NM US",0.2664,COOP:290079,METERS,-108.30000
...,...,...,...,...,...,...,...,...,...
143546,1637.7,1960-05-02,2022-06-16,32.63333,"SILVER CITY GRANT CO AIRPORT, NM US",1.0000,WBAN:93063,METERS,-108.16667
143553,2075.7,2005-01-01,2022-06-16,33.45000,"RUIDOSO SIERRA BLANCA AIRPORT, NM US",1.0000,WBAN:93083,METERS,-105.51667
143555,2185.7,2006-01-01,2022-06-16,35.88333,"LOS ALAMOS MUNICIPAL AIRPORT, NM US",1.0000,WBAN:93091,METERS,-106.28333


In [107]:
from math import pi, acos, sin, cos, sqrt, radians, asin
def haversine_distance(point_A, point_B):
    """
    Function to return the orthodromic distance between 2 points in miles
    formula for orthodromic distance is: Distance = 3963.0 * arccos[(sin(lat1) * sin(lat2)) + cos(lat1) * cos(lat2) * cos(long2 – long1)]
    Args: 
        point_A: [list] - a list containing [latitude, longitude] decimal coordinates of the first point
        point_B: [list] - a list containing [latitude, longitude] decimal coordinates of the second point
    Returns:
        distance (km)
    """
    assert type(point_A) == list, "Coordinates for point_A must be a list of [lat, long]"
    assert type(point_B) == list, "Coordinates for point_B must be a list of [lat, long]"

    # rad_A = [coord / (180 * pi) for coord in point_A]
    # rad_B = [coord / (180 * pi) for coord in point_B]

    r_earth = 6371 # radius of the earth in km
    # d = r_earth * acos((sin(rad_A[0]) * sin(rad_B[0])) + cos(rad_A[0]) * cos(rad_B[0]) * cos(rad_B[1] - rad_A[1]))
    # d = r_earth * acos((sin(point_A[0]) * sin(point_B[0])) + cos(point_A[0]) * cos(point_B[0]) * cos(point_B[1] - point_A[1]))

    [lat_A, lon_A], [lat_B, lon_B] = point_A, point_B
    lat_A, lon_A, lat_B, lon_B = map(radians, [lat_A, lon_A, lat_B, lon_B])

    d_lon = lon_B - lon_A
    d_lat = lat_B - lat_A

    a = sin(d_lat/2)**2 + cos(lat_A) * cos(lat_B) * sin(d_lon/2) ** 2
    c = 2 * asin(sqrt(a))
    d = c * r_earth
    return d

point_A = stations[['latitude','longitude']].iloc[0].to_list()
point_B = stations[['latitude','longitude']].iloc[1].to_list()

haversine_distance(point_A, point_B)

# print(point_A, point_B)


348.8485886824953

In [108]:
nm_stations['distance_to_home'] = nm_stations[["latitude", "longitude"]].apply(
    lambda x: haversine_distance(
        point_A=[my_lat, my_lon],
        point_B=[x[0], x[1]]
        ), axis=1
    )
nm_stations['elev_difference'] = nm_stations['elevation'].apply(
    lambda x: x - my_elev_m
)
nm_stations

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,distance_to_home,elev_difference
idx,,,,,,,,,,,
10973,1874.5,1931-01-01,2015-11-01,36.30270,"ABBOTT 1 SE, NM US",0.8499,COOP:290022,METERS,-104.24970,243.677245,172.5
10974,1800.1,1942-07-01,1948-04-01,36.16667,"ABIQUIU, NM US",0.9284,COOP:290039,METERS,-106.18333,116.424056,98.1
10975,1944.6,1957-06-01,2015-11-01,36.24020,"ABIQUIU DAM, NM US",0.9943,COOP:290041,METERS,-106.42770,120.169435,242.6
10976,2007.1,1941-01-01,1953-04-01,35.05000,"ACOMITA AIRPORT, NM US",0.9328,COOP:290074,METERS,-107.71667,106.264709,305.1
10977,2396.9,1948-06-01,1949-08-01,34.46667,"ADAMS DIGGINGS, NM US",0.2664,COOP:290079,METERS,-108.30000,177.038704,694.9
...,...,...,...,...,...,...,...,...,...,...,...
143546,1637.7,1960-05-02,2022-06-16,32.63333,"SILVER CITY GRANT CO AIRPORT, NM US",1.0000,WBAN:93063,METERS,-108.16667,318.255447,-64.3
143553,2075.7,2005-01-01,2022-06-16,33.45000,"RUIDOSO SIERRA BLANCA AIRPORT, NM US",1.0000,WBAN:93083,METERS,-105.51667,213.252010,373.7
143555,2185.7,2006-01-01,2022-06-16,35.88333,"LOS ALAMOS MUNICIPAL AIRPORT, NM US",1.0000,WBAN:93091,METERS,-106.28333,83.666686,483.7


In [95]:
close_stations = nm_stations.sort_values(["distance_to_home"]).iloc[:25]

In [123]:
from datetime import datetime
close_stations = nm_stations[(
    nm_stations.maxdate.dt.year >= datetime.today().year
    ) & (
    nm_stations.maxdate.dt.month >= datetime.today().month
    ) & (
    nm_stations.datacoverage >= 0.8
    )].sort_values(['distance_to_home', 'maxdate']).iloc[0:25].reset_index(drop=True)

close_stations.to_csv('info/close_stations.csv')

In [124]:
close_stations

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude,distance_to_home,elev_difference
0,1657.2,2008-08-01,2022-06-14,35.169806,"SANDIA HEIGHTS 3.7 W, NM US",1.0000,GHCND:US1NMBR0107,METERS,-106.554786,0.610708,-44.8
1,1625.5,2012-02-01,2022-06-14,35.172928,"ALBUQUERQUE 5.0 NE, NM US",0.9739,GHCND:US1NMBR0149,METERS,-106.567337,1.325345,-76.5
2,1608.7,2019-04-01,2022-06-13,35.149231,"ALBUQUERQUE 3.5 NE, NM US",0.9590,GHCND:US1NMBR0313,METERS,-106.576276,2.453207,-93.3
3,1634.0,2019-04-01,2022-06-14,35.133050,"ALBUQUERQUE 3.7 ENE, NM US",0.9997,GHCND:US1NMBR0306,METERS,-106.562800,3.548352,-68.0
4,1676.1,2005-03-01,2022-06-13,35.127832,"ALBUQUERQUE 4.6 E, NM US",1.0000,GHCND:US1NMBR0034,METERS,-106.543702,4.260704,-25.9
5,1802.0,2020-08-11,2022-06-13,35.167200,"ALBUQUERQUE 7.4 ENE, NM US",1.0000,GHCND:US1NMBR0365,METERS,-106.509500,4.319393,100.0
6,1547.8,2012-07-01,2022-06-14,35.165437,"ALBUQUERQUE 3.5 NNE, NM US",1.0000,GHCND:US1NMBR0139,METERS,-106.606399,4.499514,-154.2
7,1833.1,2020-09-16,2022-06-13,35.166502,"ALBUQUERQUE 7.7 ENE, NM US",1.0000,GHCND:US1NMBR0366,METERS,-106.502275,4.971001,131.1
8,1684.0,2013-07-01,2022-06-12,35.122353,"ALBUQUERQUE 5.0 E, NM US",1.0000,GHCND:US1NMBR0168,METERS,-106.535507,5.084387,-18.0
9,1816.0,2005-03-01,2022-06-14,35.186100,"ALBUQUERQUE 8 NE, NM US",0.9998,GHCND:US1NMBR0017,METERS,-106.507100,5.119982,114.0


In [127]:
params = {
    "locationid": "US1NMBR0107",
    "datasetid": "GHCND"
}

noaa_data.stations(params)


Metadata for stations: 
 {'resultset': {'offset': 1, 'count': 143928, 'limit': 1000}}


KeyboardInterrupt: 

In [7]:
# nm_locations = locations[locations.name.str.contains("NM")]
# nm_locations.to_csv('nm_locations.csv', index=False)

In [8]:
# locations.to_csv('all_locations.csv', index=False)

In [ ]:
# https://www.ncei.noaa.gov/cdo-web/api/v2/{endpoint}
token = 'HPlApRKyCHekmCNBeYOiPtNwWMBjybPt'
creds = dict(token=token)
dtype = 'stations'
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets'

r = requests.get(url,headers={"token":token})


In [ ]:
r.json()